In [ ]:
import pandas as pd
from langdetect import detect

def detect_en_lang(title, description):
    try:
        return detect(title) == 'en' and detect(description) == 'en'
    except:
        return False

    
cavideos = pd.read_csv('dataset/CAvideos.csv')
devideos = pd.read_csv('dataset/DEvideos.csv')
frvideos = pd.read_csv('dataset/FRvideos.csv')
gbvideos = pd.read_csv('dataset/GBvideos.csv')
usvideos = pd.read_csv('dataset/USvideos.csv')
dataset = pd.DataFrame()
dataset = dataset.append(cavideos).append(devideos).append(frvideos).append(gbvideos).append(usvideos)
# dataset = dataset[dataset.apply(lambda x: detect_en_lang(str(x['title']), str(x['description'])), axis=1)]
dataset.count()

In [28]:
from langdetect import detect
detect("War doesn't show who's right, just who's left.")

'en'

In [26]:
dataset = pd.DataFrame()
data = pd.read_csv('dataset/DEvideos.csv')
dataset = dataset.append(data)
dataset.count()

video_id                  29340
trending_date             29340
title                     29340
channel_title             29340
category_id               29340
publish_time              29340
tags                      29340
views                     29340
likes                     29340
dislikes                  29340
comment_count             29340
thumbnail_link            29340
comments_disabled         29340
ratings_disabled          29340
video_error_or_removed    29340
description               28219
dtype: int64

In [20]:
df = pd.DataFrame()
data = pd.DataFrame({"A": range(3)})
df.append(data)

,A
0,0
1,1
2,2


video_id                  29340
trending_date             29340
title                     29340
channel_title             29340
category_id               29340
publish_time              29340
tags                      29340
views                     29340
likes                     29340
dislikes                  29340
comment_count             29340
thumbnail_link            29340
comments_disabled         29340
ratings_disabled          29340
video_error_or_removed    29340
description               28219
dtype: int64